# sEEG Decoding Tutorial

## We have 84 electrodes/contacts for this current sEEG data set. It is recorded in 1024Hz. To remove the noise, and make our analysis more quick and easy, we downsampled it to 90Hz.




In [20]:
import time
import os
import numpy as np
import sys
sys.path.append("../dataset")
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from binary_label_dataset import BinaryLabelDataset

### Load the sEEG data, its shape is (number_of_electrode, samples_recorded)
### Load the face labels for character Tony. Each second consists of 30 frames. Our labeling criterion is based on the presence of Tony: if equal or more than 20 frames within a single second feature Tony, we label this second as '1'. If equal or less than 10 frames within a single second feature Tony, we label it as "0. Otherwise we drop the data.

In [24]:

seeg_file = '../../data/downsampled_seeg.npy'
label_file = '../../data/seconds_with_Tony200.npy'
#In order to compare the accuracy with DNN, we also split the data into 70% traning, 15% testing and 15% validation, although 15% validation is not used.
train_ratio = 0.7
test_ratio = 0.15
train_dataset =BinaryLabelDataset(seeg_file=seeg_file, label_file=label_file, split='train', train_ratio=train_ratio,
                                    test_ratio=test_ratio)
val_dataset = BinaryLabelDataset(seeg_file=seeg_file, label_file=label_file, split='val', train_ratio=train_ratio,
                                    test_ratio=test_ratio)
test_dataset = BinaryLabelDataset(seeg_file=seeg_file, label_file=label_file, split='test', train_ratio=train_ratio,
                                    test_ratio=test_ratio)



Initialized train dataset with 4454 samples
Initialized val dataset with 956 samples
Initialized test dataset with 954 samples


In [25]:
seeg_train, label_train = train_dataset.seeg_data, train_dataset.label_data
seeg_test, label_test = test_dataset.seeg_data, test_dataset.label_data

In [26]:
seeg_train.shape

(4454, 84, 90)

In [27]:
label_train.shape


(4454,)

In [28]:
#Reshape the data to fit the SVM pipeline
X_train = seeg_train.reshape(seeg_train.shape[0],-1)
y_train = label_train
X_test = seeg_test.reshape(seeg_test.shape[0],-1)
y_test = label_test

In [29]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Perform SVM to do the classfication between Tony and Non-Tony video segments.


In [30]:

# Train the SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5335


### Evaluate the SVM with different kernel and  c (Regularization Parameter)


In [31]:
# Train the SVM
svm_model = SVC(C=0.001, kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5042


In [32]:
# Train the SVM
svm_model = SVC(C=0.0001, kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5294


In [33]:
# Train the SVM
svm_model = SVC(C=0.001, kernel='poly')
svm_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5881


In [34]:
# Train the SVM
svm_model = SVC(C=0.0001, kernel='poly')
svm_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5881
